<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Импорт-библиотек" data-toc-modified-id="Импорт-библиотек-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Импорт библиотек</a></span></li><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Баланс-классов" data-toc-modified-id="Баланс-классов-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Баланс классов</a></span></li><li><span><a href="#Лемматизация-данных" data-toc-modified-id="Лемматизация-данных-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Лемматизация данных</a></span></li><li><span><a href="#Очистка-данных" data-toc-modified-id="Очистка-данных-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Очистка данных</a></span></li><li><span><a href="#Создание-признаков" data-toc-modified-id="Создание-признаков-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Создание признаков</a></span></li><li><span><a href="#Расчет-TF-IDF" data-toc-modified-id="Расчет-TF-IDF-1.7"><span class="toc-item-num">1.7&nbsp;&nbsp;</span>Расчет TF-IDF</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Вспомогательная-функция" data-toc-modified-id="Вспомогательная-функция-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Вспомогательная функция</a></span></li><li><span><a href="#LogisticRegression" data-toc-modified-id="LogisticRegression-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>LogisticRegression</a></span></li><li><span><a href="#CatBoostClassifier" data-toc-modified-id="CatBoostClassifier-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>CatBoostClassifier</a></span></li><li><span><a href="#DummyClassifier" data-toc-modified-id="DummyClassifier-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>DummyClassifier</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

### Импорт библиотек

In [ ]:
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np
import spacy
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
from sklearn.dummy import DummyClassifier

### Загрузка данных

In [ ]:
data = pd.read_csv('toxic_comments.csv')

### Баланс классов

In [ ]:
data['toxic'].value_counts(normalize=True)

0    0.898321
1    0.101679
Name: toxic, dtype: float64

### Лемматизация данных

In [ ]:
# функция лемматизации текста

nlp = spacy.load("en_core_web_sm")

def lemmatize(text):
    doc = nlp(text)

    return ' '.join([token.lemma_ for token in doc])

In [ ]:
%%time

data['lemm_text'] = data['text'].apply(lambda text: lemmatize(text))

CPU times: user 47min 26s, sys: 21.6 s, total: 47min 48s
Wall time: 47min 54s


In [ ]:
data.head()

,text,toxic,lemm_text
0,Explanation\nWhy the edits made under my usern...,0,Explanation \n why the edit make under my user...
1,D'aww! He matches this background colour I'm s...,0,D'aww ! he match this background colour I be s...
2,"Hey man, I'm really not trying to edit war. It...",0,"hey man , I be really not try to edit war . it..."
3,"""\nMore\nI can't make any real suggestions on ...",0,""" \n more \n I can not make any real suggestio..."
4,"You, sir, are my hero. Any chance you remember...",0,"you , sir , be my hero . any chance you rememb..."


### Очистка данных

In [ ]:
data['lemm_text'] = data['lemm_text'].str.replace(r'[^a-zA-Z ]', ' ', regex=True)\
                                     .str.split()\
                                     .str.join(' ')

In [ ]:
data.head()

,text,toxic,lemm_text
0,Explanation\nWhy the edits made under my usern...,0,Explanation why the edit make under my usernam...
1,D'aww! He matches this background colour I'm s...,0,D aww he match this background colour I be see...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man I be really not try to edit war it be ...
3,"""\nMore\nI can't make any real suggestions on ...",0,more I can not make any real suggestion on imp...
4,"You, sir, are my hero. Any chance you remember...",0,you sir be my hero any chance you remember wha...


### Создание признаков

In [ ]:
train, test = train_test_split(data, test_size=0.5, random_state=42)

corpus_train = train['lemm_text'].values
corpus_test = test['lemm_text'].values

y_train = train['toxic']
y_test = test['toxic']

### Расчет TF-IDF

In [ ]:
nltk.download('stopwords')

stopwords = set(nltk_stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
tf_idf = TfidfVectorizer(stop_words=stopwords)

tf_idf_train = tf_idf.fit_transform(corpus_train)

tf_idf_test = tf_idf.transform(corpus_test)

## Обучение

### Вспомогательная функция

In [ ]:
# функция расчета метрики f1 на кросс-валидации и тестовой выборке

def train_test_f1(model):
    pipe = Pipeline([('estimator', model)])
    
    score_cv = cross_val_score(pipe, tf_idf_train, y_train, scoring='f1', cv=3, n_jobs=-1).mean()
    pipe.fit(tf_idf_train, y_train)
    score_test = f1_score(y_test, pipe.predict(tf_idf_test))
    
    print(f'Метрика F1 на кросс-валидации: {score_cv:.2f}')
    print(f'Метрика F1 на тесте: {score_test:.2f}')

### LogisticRegression

In [ ]:
LR = LogisticRegression(max_iter=1000, class_weight='balanced')

In [ ]:
%%time

train_test_f1(LR)

Метрика F1 на кросс-валидации: 0.75
Метрика F1 на тесте: 0.74
CPU times: user 6.04 s, sys: 6.38 s, total: 12.4 s
Wall time: 12.8 s


### CatBoostClassifier

In [ ]:
CBC = CatBoostClassifier(verbose=False, 
                         random_state=42, 
                         early_stopping_rounds=200,
                         auto_class_weights='Balanced')

In [ ]:
%%time

train_test_f1(CBC)

Метрика F1 на кросс-валидации: 0.75
Метрика F1 на тесте: 0.76
CPU times: user 46min 50s, sys: 12min 13s, total: 59min 3s
Wall time: 1h 26min 52s


### DummyClassifier

In [ ]:
dummy = DummyClassifier(strategy='prior')

In [ ]:
%%time

train_test_f1(dummy)

Метрика F1 на кросс-валидации: 0.00
Метрика F1 на тесте: 0.00
CPU times: user 170 ms, sys: 199 ms, total: 368 ms
Wall time: 1.9 s


## Выводы

Значения метрики `F1` на кросс-валидации и тестовой выборке приведены в сводной таблице.

|Модель                |Размер выборки|F1 на кросс-валидации|F1 на тесте|Время CV, обучения и теста|
|:---------------------|:------------:|:-------------------:|:---------:|:------------------------:|
|LogisticRegression    |38 852        |0.75                 |0.74       |12.8 с                 |
|CatBoostClassifier    |38 852        |0.75                 |0.76       |87 мин                |
|DummyClassifier       |38 852        |0.00                 |0.00       |1.9 с                   |

Лучше значение метрики `F1` на тесте равно 0.76 для модели CatBoostClassifier. Для модели LogisticRegression значение метрики `F1` несколько ниже (0.74), однако скорость расчета намного лучше (13 секунд против 87 минут).